In [1]:
import datetime
import json
import os
import pandas as pd
import requests
import sqlalchemy

In [2]:
SRC_PATH = os.environ.get('SRC_PATH')  #  path to source code os.path.abspath('.')
DB_NAME = os.environ.get('DB_NAME')  #  [database will be one directory above source code]
IEX_KEY = os.environ.get('IEX_KEY')  # IEX api key to get market data

# Create engine
engine = sqlalchemy.create_engine(f"sqlite:///{SRC_PATH.replace('/MarketView', '')}/{DB_NAME}")

In [3]:
def get_last_price(engine):
    table_name = (datetime.datetime.now() - datetime.timedelta(hours=4)).strftime('%Y-%m-%d %H:%M:%S last')
    params = {'token':IEX_KEY}
    r = requests.get(f"https://cloud.iexapis.com/stable/tops/last", params=params)
    if r.status_code == 200:
        data = pd.DataFrame(r.json()).set_index('symbol')
        data.to_sql(table_name, engine)
    return data

In [4]:
last_df = get_last_price(engine)

In [5]:
print(f'''


Number of Securities = {len(last_df)}


''')

last_df.head()




Number of Securities = 15851





,price,size,time
symbol,,,
GBF,109.990,55,1659110492363
WBK,14.220,200,1643403588574
GBGR,16.165,1,1658250932698
PBA,37.860,49,1659125603299
SPAK,14.050,127,1659120682369


In [7]:
# get most recent [last] table by filtering tables with last in name
last_table_name = sorted(list(filter(lambda table_name: 'last' in table_name, engine.table_names())), reverse=True)[0]

print(last_table_name)

pd.read_sql(last_table_name, engine, index_col='symbol')

2022-08-01 03:36:59 last


/home/ec2-user/lab/lab_venv/lib64/python3.7/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


,price,size,time
symbol,,,
GBF,109.990,55,1659110492363
WBK,14.220,200,1643403588574
GBGR,16.165,1,1658250932698
PBA,37.860,49,1659125603299
SPAK,14.050,127,1659120682369
...,...,...,...
TEQI,34.460,4,1659117354875
BAC-K,25.380,76,1659124759081
TUGC,9.860,85,1651262398313
